In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
import glob

def merge_cuffdiff(genefile):
    spath = os.getcwd()
    gene_id = pd.read_csv(genefile)
    for root, dirs, filenames in os.walk(spath):
        for f in filenames:
            if f.endswith('gene_exp.diff'):
                x = pd.read_table(os.path.join(root, f))
                x = x.rename(columns={ x.columns[9]: "logFC" })
                y = os.path.basename(root)
                x.columns = [str(col) + '_' + y for col in x.columns]
                x = x.rename(columns={ x.columns[0]: "gene_id" })
                gene_id = gene_id.merge((x), on = 'gene_id', how = 'left')
    return gene_id

def FPKM_restriction(c):
    vals = []   
    cols = list(c.columns.values)
    for i,y in enumerate(cols):
        if 'value' in y:
            vals.append(y)
        for z in vals:    
            if 'p_value' in z:
                vals.remove(z)
            if 'q_value' in z:
                vals.remove(z)
    for q in vals:  
        c = c[c[q] >= 1] 
        merged_table = c
    return merged_table
        
def split_gene_ID(x):
    new_IDs = x
    new = new_IDs["gene_id"].str.partition(".")
    new.columns = ['gene_id','.','decimal']
    new_IDs['gene_id'] = new['gene_id']
    return new_IDs

def merge_metrics(d):
    for file in glob.glob("METRICS/*.txt"):
        file = pd.read_table(file)
        file = file.rename(columns={ file.columns[0]: "gene_id" })
        d = d.merge((file), on = 'gene_id', how = 'left')
    return d

In [11]:
c = merge_cuffdiff('geneNames.csv')
d = split_gene_ID(c)
e = merge_metrics(d)

In [12]:
e.to_csv('/Users/tyler-matheny/Desktop/new.csv')
e

,Unnamed: 0,test_id,gene_id,gene_id_PB,gene_PB,locus_PB,sample_1_PB,sample_2_PB,status_PB,value_1_PB,value_2_PB,logFC_PB,test_stat_PB,p_value_PB,q_value_PB,significant_PB,gene_id_SG,gene_SG,locus_SG,sample_1_SG,sample_2_SG,status_SG,value_1_SG,value_2_SG,logFC_SG,test_stat_SG,p_value_SG,q_value_SG,significant_SG,Unnamed: 14_SG,gene_id_nostressPEL,gene_nostressPEL,locus_nostressPEL,sample_1_nostressPEL,sample_2_nostressPEL,status_nostressPEL,value_1_nostressPEL,value_2_nostressPEL,logFC_nostressPEL,test_stat_nostressPEL,p_value_nostressPEL,q_value_nostressPEL,significant_nostressPEL,gene_id_STRESSPEL,gene_STRESSPEL,locus_STRESSPEL,sample_1_STRESSPEL,sample_2_STRESSPEL,status_STRESSPEL,value_1_STRESSPEL,value_2_STRESSPEL,logFC_STRESSPEL,test_stat_STRESSPEL,p_value_STRESSPEL,q_value_STRESSPEL,significant_STRESSPEL,gene_id_SG_molcell,gene_SG_molcell,locus_SG_molcell,sample_1_SG_molcell,sample_2_SG_molcell,status_SG_molcell,value_1_SG_molcell,value_2_SG_molcell,logFC_SG_molcell,test_stat_SG_molcell,p_value_SG_molcell,q_value_SG_molcell,significant_SG_molcell,gene_id_STRESS,gene_STRESS,locus_STRESS,sample_1_STRESS,sample_2_STRESS,status_STRESS,value_1_STRESS,value_2_STRESS,logFC_STRESS,test_stat_STRESS,p_value_STRESS,q_value_STRESS,significant_STRESS,Gene % GC content,DICER1,FKBP4,METAP2,DGCR8,EIF4G2,IGF2BP1,IGF2BP3,DDX59,GEMIN5,STAU1,EWSR1,GNL3,HNRNPA2B1,IGF2BP2,LSM11,DDX6_x,DDX3X,GRWD1,FXR1,LIN28A,EIF3G,ATXN2,EIF4A3,KHSRP,FASTKD2,GPKOW,FUS,FXR2,METTL3,FBL,EIF3D,DDX6_y,XRN2,FAM120A,EIF3A,TIAL1,CDC40,ILF3,MOV10,EIF3B,EFTUD2,PUM2,DHX30,LARP7,YBX3,DIS3L2,HLTF,GTF2F1,LARP4,KHDRBS1,TIA1,METTL14,FMR1,EIF3H,FTO,GRSF1,TARDBP,DDX55,MBNL2,DROSHA,DDX42,CAPRIN1,Transcript length (including UTRs and CDS)
0,0,ENSG00000000003.14,ENSG00000000003,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,PB,WT,OK,25.012900,52.856700,1.079410,3.278390,0.00005,0.000189,yes,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,PIC,WT,OK,9.892250,13.371300,0.434769,1.97667,0.00050,0.003402,yes,-0.208204,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,SG,PEL,OK,32.737500,28.338000,-0.208204,-0.720960,0.21385,0.452336,no,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,SG,PEL,OK,9.581010,29.532100,1.624030,6.408920,0.00005,0.000309,yes,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,SG,TOT,OK,6.283560,16.693200,1.409610,5.807020,0.00005,0.000195,yes,ENSG00000000003.14,TSPAN6,chrX:100627108-100639991,LYS,LYSU,OK,14.910100,29.297500,0.974481,3.813030,0.00005,0.000478,yes,40.40,NaN,NaN,NaN,4.0,NaN,26.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.0,146.0,NaN,NaN,9.0,1.0,111.0,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,14.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,79.0,2.0,NaN,NaN,NaN,7.0,3796.0
1,1,ENSG00000000005.5,ENSG00000000005,ENSG00000000005.5,TNMD,chrX:100584801-100599885,PB,WT,NOTEST,0.050739,0.144920,1.514080,0.000000,1.00000,1.000000,no,ENSG00000000005.5,TNMD,chrX:100584801-100599885,PIC,WT,NOTEST,0.000000,0.000000,0,0,1.00000,1.000000,no,#NAME?,ENSG00000000005.5,TNMD,chrX:100584801-100599885,SG,PEL,NOTEST,0.056039,0.000000,-inf,0.000000,1.00000,1.000000,no,ENSG00000000005.5,TNMD,chrX:100584801-100599885,SG,PEL,NOTEST,0.000000,0.051081,inf,0.000000,1.00000,1.000000,no,ENSG00000000005.5,TNMD,chrX:100584801-100599885,SG,TOT,NOTEST,0.000000,0.025257,inf,0.000000,1.00000,1.000000,no,ENSG00000000005.5,TNMD,chrX:100584801-100599885,LYS,LYSU,NOTEST,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,no,40.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339.0
2,2,ENSG00000000419.12,ENSG00000000419,ENSG00000000419.12,DPM1,chr20:50888915-50958555,PB,WT,OK,7.257210,11.312000,0.640363,0.207529,0.52535,0.619896,no,ENSG00000000419.12,DPM1,chr20:50888915-50958555,PIC,WT,OK,22.942300,51.852200,1.17639,1.6685,0.00925,0.03

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib as plt
granules = e[['logFC_SG','logFC_SG_molcell', 'value_1_SG', 'value_1_SG_molcell', 'logFC_PB']]
granules = granules.apply(pd.to_numeric, errors='coerce')
granules = granules.dropna()
granules = FPKM_restriction(granules)
a4_dims = (11.7, 8.27)
fig, ax = plt.pyplot.subplots(figsize=a4_dims)
sns.scatterplot('logFC_SG', 'logFC_SG_molcell', hue = 'logFC_PB', palette= "coolwarm", data = granules)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
metric_cols = e.columns[56:]
metric_cols
f = e[metric_cols]
f = f.fillna(0)
X_metrics = f

y_FC = e['logFC_STRESSPEL']

X_train, X_test, y_train, y_test = train_test_split(X_metrics, y_FC,
                                                   random_state = 0)
linreg = LinearRegression().fit(X_train, y_train)

print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

est = sm.OLS(y_FC.astype(float), X_metrics.astype(float))
est2 = est.fit()
print(est2.summary())

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

np.set_printoptions(precision=2)



target_names_fruits = ['apple', 'mandarin', 'orange', 'lemon']

X_fruits_2d = fruits[['height', 'width']]
y_fruits_2d = fruits['fruit_label']

X_train, X_test, y_train, y_test = train_test_split(X_fruits, y_fruits, random_state=0)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_scaled, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train_scaled, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test_scaled, y_test)))

example_fruit = [[5.5, 2.2, 10, 0.70]]
example_fruit_scaled = scaler.transform(example_fruit)
print('Predicted fruit type for ', example_fruit, ' is ', 
          target_names_fruits[knn.predict(example_fruit_scaled)[0]-1])

In [ ]:
metric_cols

In [ ]:
vals = []   
cols = list(c.columns.values)
for i,y in enumerate(cols):
    if 'value' in y:
        vals.append(y)
    for z in vals:    
        if 'p_value' in z:
            vals.remove(z)
        if 'q_value' in z:
            vals.remove(z)
for q in vals:  
    c = c[c[q] >= 1] 
    merged_table = c


In [ ]:
for file in glob.glob("METRICS/*.txt"):
    file = pd.read_table(file)
    file = file.rename(columns={ file.columns[0]: "gene_id" })
    d = d.merge((file), on = 'gene_id', how = 'left')
for file in glob.glob("METRICS/*.txt"):
    file = pd.read_table(file)
    file = file.rename(columns={ file.columns[0]: "gene_id" })
    d = d.merge((file), on = 'gene_id', how = 'left')
        #print('file found')

In [ ]:
e.to_csv('/Users/tyler-matheny/Desktop/merged.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
sg = pd.read_csv('sgtranscriptome.csv')
sg = sg.rename(index=str, columns={"test_id": "gene_id"})
sg = sg.dropna()
sg = sg.drop(labels='Mito', axis=1)
for file in glob.glob("METRICS/*.txt"):
    file = pd.read_table(file)
    file = file.rename(columns={ file.columns[0]: "gene_id" })
    sg = sg.merge((file), on = 'gene_id', how = 'left')
sg = sg.fillna(0)
metric_cols = sg.columns[13:-1]
X_metrics = sg[metric_cols]
y_FC = sg['Fold change']
y_Loc = sg['Localization']

X_train, X_test, y_train, y_test = train_test_split(X_metrics, y_Loc,
                                                   random_state = 0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 25)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

#Dummy Classifier
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test)

y_dummy_predictions
dummy = dummy_majority.score(X_train, y_train)
print(dummy)
dummy = dummy_majority.score(X_test, y_test)
print(dummy)




#kNN Regression
from sklearn.neighbors import KNeighborsRegressor

X_train, X_test, y_train, y_test = train_test_split(X_metrics, y_FC, random_state = 0)

knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)

print(knnreg.predict(X_test))
print('R-squared kNN test score: {:.3f}'
     .format(knnreg.score(X_test, y_test)))

X_train, X_test, y_train, y_test = train_test_split(X_metrics, y_FC,
                                                   random_state = 0)

linreg = LinearRegression().fit(X_train, y_train)
print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))


est = sm.OLS(y_FC, X_metrics)
est2 = est.fit()
print(est2.summary())



Accuracy of K-NN classifier on training set: 0.76
Accuracy of K-NN classifier on test set: 0.75
0.6890627982439397
0.6886090440755581
[0.4392 1.6628 0.8482 ... 0.8758 1.4482 2.0486]
R-squared kNN test score: 0.403
linear model intercept: 0.22229704121838445
linear model coeff:
[-8.81877692e-03  6.07898352e-05  5.75217148e-05  3.05318765e-05
  2.58655860e-04  4.42308638e-02 -3.92994526e-02 -5.47446861e-03
  1.11466000e-03 -3.74807604e-04  1.72449131e-04 -2.21833163e-04
 -3.72586028e-04  6.30399122e-04 -3.07172937e-04 -6.63633310e-04
 -5.04566924e-05 -6.13439410e-05 -1.14790566e-02  1.48806497e-03
  7.78079894e-04 -1.81983598e-03  1.60535476e-03 -5.08341186e-05
  5.98980708e-04 -3.86463724e-03 -1.98682133e-03 -1.14786990e-03
 -5.13600019e-04 -3.41526372e-03 -3.11384610e-06 -3.31413184e-04
  3.03201227e-03 -2.30414697e-04  7.72207811e-04 -1.60675821e-03
 -3.05783416e-03 -1.46322353e-03  1.60535476e-03 -6.50511273e-04
  7.94630163e-05  1.19176554e-03  3.59079881e-04  6.31203613e-04
 -2.237

In [ ]:
sg

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

np.set_printoptions(precision=2)

X_metrics = sg[metric_cols]
y_FC = sg['Localization']

target_names = ['Nether', 'enriched', 'depleted']

X_train, X_test, y_train, y_test = train_test_split(X_metrics, y_FC, random_state=0)

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
#X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train_scaled, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test_scaled, y_test)))

example_fruit = [[5.5, 2.2, 10, 0.70]]
example_fruit_scaled = scaler.transform(example_fruit)
print('Predicted fruit type for ', example_fruit, ' is ', 
          target_names_fruits[knn.predict(example_fruit_scaled)[0]-1])

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3)
model.fit(X_metrics)


In [ ]:
# Importing Modules
from sklearn import datasets
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt



# Defining Model
model = TSNE(learning_rate=100)

# Fitting Model
transformed = model.fit_transform(X_metrics)

# Plotting 2d t-Sne
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

plt.scatter(x_axis, y_axis)
plt.show()

In [ ]:
# Load Python Libraries
import swat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
%matplotlib inline



In [ ]:
rndperm = np.random.permutation(X_metrics.shape[0])

In [ ]:
rndperm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


# Plot the graph
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,30):
    ax = fig.add_subplot(3,10,i+1)
    ax.matshow(X_metrics.loc[rndperm[i]])
plt.show()